___

<a href='https://www.learntocodeonline.com/'> <img src='files/IMGs/learn to code online.png' /></a>
___

# [Create Login API ViewSet](https://www.udemy.com/course/django-python/learn/lecture/6955102#questions)

This API will allow users to log into the system using **Token Authentication**. It works by generating a token (random string) sent to the API for every request you make that requires authentication. This section will walk through how to generate the token - or create a login API.

These changes will be made in the `views.py` file in the **profiles_api** app folder. Once again, the Django platform has something that can do this for you - but it's fixed as an API view and doesn't have a ViewSet option to use.

You therefore cannot use it with your standard default router used to setup standard URLs. So we have to *trick the system* to think it's a ViewSet.

1. Import the auth token that comes with the Django REST framework & the API settings

In [ ]:
# from rest_framework.authtoken.serializers import AuthTokenSerializer
from rest_framework.authtoken.views import ObtainAuthToken
from rest_framework.settings import api_settings

2. Create an API ViewSet which passes the request through to the obtained AuthToken APIView. Then you will have a ViewSet for your API login that you canuse with your default router.

You will now create a new class ... The **ObtainAuthToken** does not enable itself by default in the browsable Django admin site - so it needs to be overwritten.

In [ ]:
class UserLoginApiView(ObtainAuthToken):
    """Handle creating user authentication tokens"""
    
    # add renderer classes to ObtainAuthToken view
    # enabled in the Django admin
    renderer_classes = api_settings.DEFAULT_RENDERER_CLASSES

In [ ]:
# this was from 2018 ---- not used when reviewing 2022 training
class LoginViewSet(viewsets.ViewSet):
    """Checks email and password & returns an auth token."""
    
    serializer_class = AuthTokenSerializer
    
    # create a CREATE function (HTTP POST to ViewSet)
    def create(self, request):
        """User the ObtainAuthToken APIView to validate and create a token."""
        
        # pass the request through to ObtainAuthToken APIView
        # and call the POST request - you create a new isntance
        # of the ObtainAuthToken by adding () ...
        # It does the same thing it would normally, but we're
        # calling it through our ViewSet & return the response
        # in the CREATE function.
        return ObtainAuthToken().post(request)

## 3. Add the ViewSet to your URL router.

This is located in the `urls.py` file in the **profiles_api** app folder.

You will need to create a new router underneat the profile router.

As of 2018, you would do so by adding a new router:  `router.register('login', views.LoginViewSet, base_name='login')`

    Since it's not a **models** ViewSet we need to set the **base_name** in the call. Otherwise there is no way for Django to know what to call this object our router.

But as of 2022, you would update the **urlpatterns** with an additional path between the `hello-view` and `''` paths:
`path('login/', views.UserLoginApiView.as_view())`

<img src='files/IMGs/views/10-Login-URL.png'>

# Test Login API

Follow the same process as before:

1. ensure the vagrant server is up
- open git bash
- `vagrant up`
- `workon profiles_api`
- `cd /vagrant/src/profiles_project`
- `python manage.py runserver 0.0.0.0:8000`

2. access the API in your Chrome browser on the base API URL:  **http://127.0.0.1:8000/api/login**

As per 2022, you cannot see this from the root. If we wanted to, it would need to be created as a ViewSet instead of an APIView because then it could be viewed from root:

<img src='files/IMGs/login-01.png'>

3. Click the login URL from root.

When you get to the page, it has done an HTTP GET, which returns an HTTP 405 "method not allowed" response. That's because you can't make this method call on a login API - only a POST to log in.

<img src='files/IMGs/login-02.png'>

4. Log in as your super user. (Remember that username is the email address in this example, since we used the built-in serializer of the REST framework.)

The API will then post the username and password to the server and generate a temporary token used to authenticate with the other APIs.

<img src='files/IMGs/login-03.png'>

This token will be used in the next portion, so be sure to keep it handy.

# Set Token Header Using ModHeader Extension

This section will show you how to use your AuthToken to authorize yourself with your API.

Here's how it works in a real app:
- client side app will make a HTTP request to our login endpoint with the username and password it wants to authenticate
- a user will enter their email & password
- the app will call the API login endpoint
- backend will confirm that the username / password are valid
- if valid will generate and return a token to the user
- the app is then responsible for ensuring the token is included in all future HTTP requests

The app does this by adding the token to the HTTP headers for all future requests it makes. The HTTP header is like metadata that goes along with the HTTP request. (Data about the request.)

In that data is an authorization HTTP header. This authorization HTTP header will include your token. Then every time the app makes a request to the API, we can check it it's provided a valid token in the HTTP headers. If valid, it will perform the request. Otherwise will return a 401 unauthorized response.

The app can then redirect the user back to the login page to get a new token.

## Using ModHeader Extension

Here's how you simulate adding the HTTPS headers to your request using **ModHeaders** - a chrome extension.

This extension will be needed for the rest of the course. It may be blue, mine was grey.

<img src='files/IMGs/ModHeaders-01.png'>

When you click on it, you will receive a nice window you can use to add custom HTTP headers to yor request.

<img src='files/IMGs/ModHeaders-02.png'>

Before adding any headers, you need to add a filter. This filter will ensure HTTP headers only apply to this specific web page. The reason for this is if you forget this is still running, it can interfere with other sites.

1. Click on the plus symbol and choose **Filter** to add a new filter.

?

2. Add a filter that matches any website that contains the URL:  **127.0.0.1:8080**

<img src='files/IMGs/ModHeaders-03.png'>

<img src='files/IMGs/ModHeaders-04.png'>

You will use the following code:  `*//127.0.0.1:8080/*`

The __\*__ is a wildcard - meaning it will match any text before or after, but will match all the other pieces that are not a wildcard.

<img src='files/IMGs/ModHeaders-05.png'>

3. Add your authorization header in the **Request Headers** section.

`Authorization` will be th name of the header you're adding.

<img src='files/IMGs/ModHeaders-06.png'>

In the value section, add `Token` then a space, and then the token you received in prior ObtainAuthToken request.

<img src='files/IMGs/ModHeaders-07.png'>

4. Test in your API by doing the following:
- go back to API root
- click on the profile page

<img src='files/IMGs/ModHeaders-08.png'>

You should now see that you are logged in as your Super User.

<img src='files/IMGs/ModHeaders-09.png'>

Add the ID to the end of the URL to see the detailed view.

<img src='files/IMGs/ModHeaders-10.png'>

You can see that you now have the options to update and delete the user account.

5. Test the option to edit the user.

Modify the name. Be sure to use the correct password. Then click PUT button.

<img src='files/IMGs/ModHeaders-11.png'>

You can see the name now has a 1 at the end.

## How can you log out?

Go back into **ModHeaders** and uncheck the authorization header & go back into the address bar then hit enter.

**DO NOT** refresh the page, as it will attempt the POST again.

<img src='files/IMGs/ModHeaders-13.png'>

You should see that the edit option has disappeared since you are no longer authorized with the API.

Turn authorization back on, then go to your user profile list.

_What happens when you try to access another profile that is **not** the one you are logged in as?_

Remember - you have a permissions class that says you can only modify the user you are logged in as.

## Commit To Git

In your **git bash** program ...

1. go to project directory:  `cd workspace/PROJECTNAME` (in this example **profiles-rest-api**)
2. Call `git add .`
3. Call `git commit -am "Added login API."`